## Pydantic fundamentals

In [1]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name = "Andreas", age = 33, gender= "M")
person1

In [4]:
person1.name, person1.age

('Andreas', 33)

In [5]:
person2 = Person(name = 3213+3213, gender= True, age="minus ten")
person2

## Validation of our Person class

In [7]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

try:
    person1 = Person(name = 3.1415, age = 33, gender= "M")

except TypeError as err:
    print(err)

Name must be of type str not <class 'float'>


In [ ]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender

        if not isinstance(age, int):
            raise TypeError(f"age must be of type int not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self.age = age

    
    
    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"
try:
    person3 = Person(name = "Ragnar", age = -54, gender= "M")

except ValueError as err:
    print(err)

age must be between 0 and 124 not -54


In [12]:
person4 = Person(name = "Bella", age = 4, gender= "F")
person4

Person(Bella, F, 4)

In [13]:
person4.age = -5
person4

Person(Bella, F, -5)

In [14]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if not isinstance(age, int):
            raise TypeError(f"age must be of type int not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self._age = age

    
    
    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"

person5 = Person(name = "Nataly", age = 4, gender= "F")
person5

Person(Nataly, F, 4)

In [17]:
try:
    person5.age = 125
except ValueError as err:
    print(err)

age must be between 0 and 124 not 125


In [18]:
try:
    Person(name= "Nataly", age = "4", gender = "F")
except TypeError as err:
    print(err)

age must be of type int not <class 'str'>


## Validate using pydantic

In [19]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int

person6 = Person(name = "Christina", gender = "F", age = 29)
person6

Person(name='Christina', gender='F', age=29)

In [21]:
person6.age = 36
person6

Person(name='Christina', gender='F', age=36)

In [ ]:
#issue
person6.age = "thirty"
person6

Person(name='Christina', gender='F', age='thirty')

In [25]:
from pydantic import ValidationError
try:
    Person(name = 834921, gender = "F", age = 29)
except ValidationError as err:
    print(err)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=834921, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [26]:
Person(name = "Nina", gender = "F", age = "29")

Person(name='Nina', gender='F', age=29)

In [30]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age: int

    model_config = ConfigDict(validate_assignment=True)

Person7 = Person(name= "Nina", gender = "F", age = "29")
try:    
    Person7.age = "thirty"
except ValidationError as err:
    print(err)

Person7.age = 10

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [31]:
try:
    Person(name = 3.1415, gender = 2.714, age = -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=3.1415, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=2.714, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [32]:
try:
    Person(name = "Sofia", gender = "F", age = -4)
except ValidationError as err:
    print(err)

## Add age validation

In [35]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt = -1, lt = 125)

    model_config = ConfigDict(validate_assignment=True)

try:
    Person(name = "Sofia", gender = "Fem", age = -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='Fem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Serilization and deserialzation

In [ ]:
#Serialzitaion
Person7.model_dump()

{'name': 'Nina', 'gender': 'F', 'age': 10}

In [ ]:
import json

with open("person.json", "w") as file:
    json.dump(Person7.model_dump(), file)



In [42]:
# deserialziation

with open("person.json", "r") as file:
    json_data = file.read()

print(repr(json_data))

Person.model_validate_json(json_data)

'{"name": "Nina", "gender": "F", "age": 10}'


Person(name='Nina', gender='F', age=10)